In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from IPython.display import Image, display, clear_output
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from pathlib import Path
import matplotlib.pyplot as plt
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D
import scipy.io
import pandas as pd
from PIL import Image, ImageOps
# %matplotlib notebook

# get rid of uninteresting warning
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from sklearn.cluster import KMeans


matplotlib_axes_logger.setLevel('ERROR')


In [3]:
from autoencoder import *
import torch

dataset = UTKFaceDataset()

encoder = Encoder(encoded_space_dim=256)
decoder = Decoder(encoded_space_dim=256)

encoder_weights = torch.load("models/encoder_256.pt", map_location=torch.device('cuda'))
decoder_weights = torch.load("models/decoder_256.pt", map_location=torch.device('cuda'))

encoder.load_state_dict(encoder_weights)
decoder.load_state_dict(decoder_weights)

100%|██████████| 23705/23705 [01:25<00:00, 278.77it/s]


<All keys matched successfully>

In [5]:
encoder.eval()
decoder.eval()

Decoder(
  (decoder_lin): Sequential(
    (0): Linear(in_features=256, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
  )
  (unflatten): Unflatten(dim=1, unflattened_size=(64, 8, 8))
  (decoder_conv): Sequential(
    (0): ConvTranspose2d(64, 32, kernel_size=(9, 9), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(32, 16, kernel_size=(9, 9), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(16, 8, kernel_size=(9, 9), stride=(2, 2), padding=(3, 3), output_padding=(1, 1))
    (7): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(8, 3, kernel_size=(9, 9), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
 

In [21]:
Z = np.load("encoded_data.npy")
import pandas as pd
df = pd.read_csv("data/info.csv")

In [25]:
y = df.iloc[:,2:5].to_numpy()
y

array([[100,   0,   0],
       [100,   0,   0],
       [100,   1,   0],
       ...,
       [  9,   1,   4],
       [  9,   1,   4],
       [  9,   1,   4]])

In [28]:
import numpy as np
from sklearn import metrics

def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 


In [32]:

kmeans = KMeans(n_clusters=16)
labels = kmeans.fit_predict(Z)
print(purity_score(y[:,1],labels))
plt.imshow(
    metrics.cluster.contingency_matrix(y[:,1], labels),
    interpolation="nearest",cmap="Reds")

0.6298249314490614
